<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Многомерное интегрирование методом Монте-Карло #

### Modules - Numerical Integration
<section class="post-meta">
By Tor Nordam
</section>
___
Этот блокнот даст краткое введение в $D$-мерное численное интегрирование, сравнивая два наивных метода:
  * Сумма Римана
  * Метод Монте-Карло с равномерной выборкой
 
## Сумма Римана ##
Сумма Римана, пожалуй, является самой простой и интуитивно понятной схемой численного интегрирования. Вы хотите интегрировать
функцию на интервале $x$. Разделите свой интервал на $N$ подинтервалов равной длины $\Delta x = L/N$.
Оцените функцию в середине каждого подинтервала. Вклад в общую площадь от субинтевала
со средней точкой $x_i$ тогда равен $f(x_i) \Delta x$, а общее значение интеграла равно
$$
\Delta x\sum_{i=1}^{N} f(x_i).
$$
Обобщение на $D$ измерений весьма тривиально. Вместо того, чтобы иметь N равномерно расположенных точек вдоль линии, вы создаете $D$-мерную сетку точек с равным расстоянием, $\Delta x$, во всех измерениях. Вклад в общий интеграл от объема с центром $\mathbf{x}_i$ is $f(\mathbf{x}_i) \cdot \Delta x^D$.

## Интегрирование методом Монте-Карло с равномерной выборкой ##

Простейшая схема интегрирования Монте-Карло по существу очень похожа на метод суммы Римана, за исключением того, что точки, в которых должна оцениваться функция, выбираются случайным образом, а не на равном расстоянии друг от друга. Равномерная выборка означает, что каждая точка имеет одинаковую вероятность выбора. Более продвинутые версии Монте-Карло могут, например, использовать более плотную выборку в областях, где значение функции быстро меняется (*т.е.*, где производная функции велика).

Для большого числа случайных точек интеграл, полученный методом Монте-Карло, приблизится к истинному значению интеграла.

Обобщение на измерения $D$ естественно следует и в этом случае. Вместо того, чтобы выбирать случайную точку вдоль линии, мы выбираем случайную точку в измерениях $D$, что равносильно генерации случайных чисел $D$ в правильных диапазонах и принятию их в качестве координат точки.

In [ ]:
# Эти строки импортируют библиотеку numpy, настраивают matplotlib
# для использования непосредственно в блокноте
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

Для аналитического решения нам понадобится гамма-функция. См. http://en.wikipedia.org/wiki/Gamma_function.

In [ ]:
from math import gamma

## Объем $D$-мерной единичной сферы  ##

Задача, которую мы будем использовать в качестве примера, состоит в том, чтобы определить объем единичной сферы, т.е. сферы с радиусом 1, в $D$ измерениях. Ответ известен аналитически и является
$$
V(r, D) = \frac{\pi^{D/2}}{\Gamma(D/2+1)}r^{D},
$$
или, для $r=1$,
$$
V(r, D) = \frac{\pi^{D/2}}{\Gamma(D/2+1)},
$$
( http://en.wikipedia.org/wiki/Volume_of_an_n-ball )

In [ ]:
# Эта функция возвращает 1, если точка находится внутри единичной сферы, 0 в противном случае.
# Обозначение *X позволяет функции принимать переменное число аргументов,
# которые доступны в виде кортежа внутри функции. Путем преобразования кортежа
# для массива numpy мы можем рассматривать его как D-мерный вектор и найти его 
# длину, взяв корень из суммы квадратов.
def unitSphere(*X):
    R = 1
    r = 0
    X = np.array(X)
    r = np.sqrt(np.sum(X**2))
    if r < R:
        return 1
    else:
        return 0

# Эта строка выполняет некоторую магию numpy для функции, позволяя ей работать
# элементарно при вызове с массивом в качестве аргумента. Таким образом, мы можем
# например, вызовать эту функцию с массивом ранга 3 и интерпретировать ее
# как массив векторов ранга 2.
unitSphere = np.vectorize(unitSphere)

# Эта функция возвращает объем единичной сферы (r=1) в d измерениях
def v(D):
    return np.pi**(D/2.0) / gamma(1+D/2.0)

In [ ]:
# Этот фрагмент кода отображает единичную сферу в 2 измерениях.
# Конечно, также можно построить график в 3 измерениях, но 
#  код matplotlib становится немного более сложным.
X = np.arange(-1, 1, 0.01)
Y = np.arange(-1, 1, 0.01)
X, Y = np.meshgrid(X, Y)
plt.clf();
plt.figure(figsize = (6,6));
plt.contourf(X, Y, unitSphere(X, Y));

## Число точек ##

С помощью метода Монте-Карло легко использовать любое количество точек, которое вам нравится, но с суммой Римана вы ограничены обычной сеткой. Для простоты давайте придерживаться одинакового расстояния в каждом направлении. Допустим, мы хотим интегрировать от -1 до 1 по направлению $x$ и разделить этот интервал на подинтервалы $N_x$. Выполнение того же самого по другим измерениям дает в общей сложности $N=N_x^D$ точек. Чтобы все было хорошо и сопоставимо, мы будем использовать одинаковое количество точек в обоих методах и писать обе функции так, чтобы они принимали $N_x$ в качестве аргумента и вычисляли общую сумму из этого.

## Суммы Римана ##

In [ ]:
# Эта функция принимает в качестве аргументов
# f - функция, подлежащая интегрированию
# Nx - количество точек вдоль каждого измерения регулярной сетки
# D - количество измерений
# start - начало интервала интегрирования (одинаково для всех измерений)
# stop - конец интервала интегрирования (одинаковый для всех измерений)
def riemannSum(f, Nx, D, start, stop):
    # шаг
    dx = (stop - start)/float(Nx)
    # Вычислит координаты точек сетки вдоль одного измерения
    # Координаты для других измерений будут одинаковыми
    x = np.linspace(start+dx/2, stop-dx/2, Nx)
    # Вычислит объем, окружающий каждую точку сетки
    vol = dx**D
    # Одномерный случай обрабатывается специально, потому что np.meshgrid
    # не работает только с одним аргументом
    if D == 1:
        points = [x]
    else:
    # По сути,
    # эта строка присваивает переменной массив ранга 2 размера D х N
    # Каждый из N столбцов представляет собой вектор D, который содержит
    # координаты точки в сетке.
        points =  np.array(np.meshgrid(*[x]*D)).reshape(D, -1)
    # Здесь f вызывается в каждой точке сетки, и сумма умножается на объем
    # каждой ячейки 
    return np.sum(f(*points))*vol


In [ ]:
# Эта функция принимает в качестве аргументов
# f - функция, подлежащая интегрированию
# Nx - количество точек вдоль каждого измерения регулярной сетки
# D - количество измерений
# start - начало интервала интегрирования (одинаково для всех измерений)
# stop - конец интервала интегрирования (одинаковый для всех измерений)
def monteCarloIntegrator(f, Nx, D, start, stop):
    # Подсчитывает общее количество точек
    N = Nx**D
    # Вычислит объем как сумму объемов ячеек
    # легко убедиться, что это то же самое, что и объем, используемый в
    # Функции суммы Римана.
    vol = (float(stop - start)**D) / N
    # Создает массив ранга 2 размера D х N, заполненный равномерно распределенными
    # случайными числами между start и stop. Каждый из N столбцов представляет собой
    # D вектор, который содержит координаты одной из случайных точек.
    points = np.random.random((D, N))*(stop - start) + start
    # Здесь f вызывается в каждой точке сетки, а сумма умножается на объем
    # каждой ячейки 
    return np.sum(f(*points))*vol

## Приложение ##

Трудно интуитивно чувствовать вещи во многих измерениях. Поэтому мы выбираем максимальное количество точек, которые мы хотим использовать, и вычисляем максимально возможное $N_x$, которое подходит.

In [ ]:
Nmax = 500000                # Верхний предел по количеству точек
D = 8                        # Количество измерений
start = -1                   # Начало интервала интегрирования (одинаковое для всех измерений)
stop  = 1                    # Конец интервала интегрирования (одинаковый для всех измерений)
Nx = int(Nmax**(1.0/D))      # Вычислит максимальное количество точек вдоль каждого измерения
N  = Nx**D                   # Рассчитает общее количество использованных точек
print("Nx:           ", Nx)
print("N:            ", N)
print("Analytic:     ", v(D))
print("Monte Carlo:  ", monteCarloIntegrator(unitSphere, Nx, D, start, stop))
print("Riemann Sum:  ", riemannSum(unitSphere, Nx, D, start, stop))

## Результаты ##
Немного поэкспериментировав с разными числами, вы должны понять пару вещей
 * Если количество измерений невелико, оба эти метода работают нормально
 * Если количество измерений велико, то общее количество точек растет чрезвычайно быстро. $N_x$
 * Если число измерений велико (например, 8 или 10), метод Монте-Карло работает лучше

Так почему же метод Монте-Карло работает лучше? В данном конкретном случае (интегрирование функции, которая зависит только от $r$), я думаю, это связано с тем, что с регулярной сеткой будет большое количество точек, которые имеют одинаковое расстояние от начала координат. Таким образом, регулировка разрешения приведет к одновременному перемещению большого количества точек внутри или за пределами единичной сферы, в результате чего сумма Римана в большинстве случаев будет завышена или занижена. С помощью метода Монте-Карло точки будут иметь лучшее распределение с точки зрения расстояния от начала координат.

Опять же, этот пример был выбран специально, чтобы показать, что метод Монте-Карло может работать лучше, чем сумма Римана, но общий момент заключающийся в том, что методы Монте-Карло, как правило, работают лучше в более высоких измерениях, сохраняется.

## Приложение ##

In [ ]:
points =  np.array(np.meshgrid(*[x]*D)).reshape(D, -1)

Что на самом деле делает эта строка кода? Давайте посмотрим на это по частям.

`np.meshgrid` принимает массивы $n$ в качестве аргументов (где $n > 1$) и возвращает массивы, которые изменяются только в одном направлении.
Это легче всего понять на примере. Допустим, у вас есть два вектора, $X$ и $Y$:

In [ ]:
X = np.arange(0, 5)
Y = np.arange(5, 10)
print("X = ", X)
print("Y = ", Y)

Затем, предположим, вы хотите создать сетку точек, например, для построения графика,
где координаты $x$ задаются $X$, а координаты $y$ - $Y$,
так что первая точка будет $(0, 5)$, вторая точка $(1,5)$, ...
шестая точка $(0, 6)$ и т.д.
Этому способствуют ``np.meshgrid``:

In [ ]:
Xs, Ys = np.meshgrid(X, Y)
print("Xs = ", Xs)
print("Ys = ", Ys)

При объединении одного элемента из массива $Xs$ с соответствующим
элементом из массива $Ys$ мы получаем координаты каждой точки.
Хотя, возможно, не сразу очевидно, это также обобщается более чем на
два измерения. В нашем случае мы создаем массив $x$ точек в
направлении $x$, который мы хотим использовать во всех направлениях. В измерениях $D$
мы можем достичь этого, вызвав meshgrid для копий $D$ массива $x$.
В Python список, умноженный на число $n$, представляет собой список, содержащий элементы
исходного списка, повторенные $n$ раз:

In [ ]:
print([1,2,3] * 3)

Обратите внимание, что это сильно отличается от умножения массива numpy на число:

In [ ]:
print(np.array([1,2,3]) * 3)

Таким образом, код `[x]*D` составляет список элементов $D$, где каждый элемент является
массивом $x$:

In [ ]:
x = np.array([1,2])
print([x]*2)

Код `np.meshgrid(*[x]*D)` использует `*` для "распаковки" списка, созданного `[x]*D`.
таким образом, эффективно вызывается функция meshgrid с аргументами $D$,
каждый из которых является копией массива $x$
``np.meshgrid(*[x]*3)``
это точно то же, что и 
``np.meshgrid(x, x, x)``.

Meshgrid возвращает кортеж массивов $D$. Результат можно превратить в массив
массивов, который совпадает с массивом одного более высокого ранга. Опять же,
пример, вероятно, самый простой:

In [ ]:
# Создание массива ранга 1
a = np.array([1,2])
# Создание кортежа из двух массивов ранга 1
b = (a, a)
print(b)
# Превращение кортежа в массив ранга 2
print(np.array(b))

Так, наконец, то строка создает массив ранга $D+1$, который
формируется в массив ранга 2 размера $(р, n)$, где $D$ - это количество измерений
и $N$ - число точек, и где каждый столбец - это $d$-вектор, который дает 
координаты одной точки.

Для небольших чисел можно проверить, что каждый столбец
действительно дает координаты уникальной точки:

In [ ]:
D = 3
x = np.array([0,1,2])
points =  np.array(np.meshgrid(*[x]*D)).reshape(D, -1)
print(points)